In [1]:
from plot_stuff import plot_BEV, plot_lidar, draw_gt_boxes3d
from loss.box_utils import point_form_3D, point_form_fafe
from label import get_labels, reshape_labels
from config import InputConfig
import mayavi.mlab
import pykitti
import numpy as np
import kitti_dataset
import time
import os

In [2]:
INPUT_CONFIG = {'x_min': 0,
                'x_max': 70,
                'x_grid_size': 0.2,
                'y_min': -40,
                'y_max': 40,
                'y_grid_size': 0.2,
                'z_min': -3,
                'z_max': 1,
                'z_grid_size': 0.2,
                'num_point_features': 3,
                'num_conseq_frames': 5,
                'num_reg_targets': 6,
                'num_anchors': 5,
                'num_classes': 1,
                'dim_gt_targets': 6,
                'max_targets_forever': 50,
                'lambda_smoothL1': 0.9,
                'alpha_factor': 0.5,
                'gamma': 2,
                'beta': 0.1,
                'anchors': [[1, 2],
                            [1, 3],
                            [2, 3],
                            [2, 4],
                            [2, 5]]}

mayavi.mlab.options.offscreen = False

In [6]:
root_dir = '/Users/erikbohnsack/data'
sequence = 10
frame = 5
input_config = InputConfig(INPUT_CONFIG)
label_path = os.path.join(root_dir, 'training', 'label_2', str(sequence).zfill(4) + '.txt')
labels, max_frame_idx = get_labels(label_path)
reshaped_labels = reshape_labels(labels, input_config)
print(reshaped_labels.shape)
non_zero_gt_mask = reshaped_labels[frame, ...].sum(dim=1) != 0
print(non_zero_gt_mask.shape)
frame_labels = reshaped_labels[frame, non_zero_gt_mask]
print(frame_labels.shape)

# to 2d point form
boxes = point_form_fafe(frame_labels)

z_center = - 1
z_height = 1.5
# to 3d point form
boxes_3d = point_form_3D(boxes, z_center, z_height)

fig = plot_lidar(root_dir, sequence, frame, split='training')
fig = draw_gt_boxes3d(boxes_3d, fig)
filename_top='testplot_top.png'
filename='testplot.png'
mayavi.mlab.view(azimuth=90, elevation=0, focalpoint=[12.0909996, -1.04700089, -2.03249991], distance=82.0, figure=fig)
mayavi.mlab.savefig(filename_top, figure=fig)
mayavi.mlab.view(azimuth=180, elevation=70, focalpoint=[12.0909996, -1.04700089, -2.03249991], distance=82.0, figure=fig)
mayavi.mlab.savefig(filename, figure=fig)

print("Saved ", filename)



torch.Size([294, 50, 6])
torch.Size([50])
torch.Size([3, 6])
Saved  testplot.png


In [ ]:
conf = InputConfig(INPUT_CONFIG)
basepath = "/Users/erikbohnsack/data/training"

tracking = pykitti.tracking(base_path=basepath, sequence="0014")
points = tracking.get_velo(3)
bev_map = np.zeros(conf.bev_shape)
print("BEV map size: {}".format(bev_map.shape))

grid_limits = np.array([[conf.x_min, conf.x_max],
                        [conf.y_min, conf.y_max],
                        [conf.z_min, conf.z_max]])
grid_sizes = np.array([conf.x_grid_size, conf.y_grid_size, conf.z_grid_size])

start_jit = time.time()
kitti_dataset.format_to_BEV_jit(bev_map, points, grid_limits, grid_sizes)
end_jit = time.time()
print("Elapsed time: {}".format(end_jit - start_jit))

for i in range(bev_map.shape[2]):
    print("Plot height channel: {}".format(i))
    print(type(bev_map))
    print(np.expand_dims(bev_map[:,:,i], 2).shape)
    plot_BEV(np.expand_dims(bev_map[:,:,i], 2))
